# Parser 시스템 테스트

새로 구현한 분리된 파서 시스템을 테스트합니다.
- PDF 파서
- DOCX 파서  
- XLSX 파서
- CSV 파서 (추가 구현 필요시)

In [1]:
# 환경 설정
import os
import sys
from pathlib import Path

# 프로젝트 루트를 Python path에 추가
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"프로젝트 루트: {project_root}")

프로젝트 루트: /Users/kimjunghyeon/Desktop/workspace/ai-agent


In [2]:
# 환경변수 로드
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

# Parser 시스템 import
from app.infra.parsers import ParserFactory
from app.services.document_processor import DocumentProcessor
import asyncio

/Users/kimjunghyeon/Desktop/workspace/ai-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 테스트 파일 경로 확인
data_dir = project_root / "data" / "documents"
print(f"데이터 디렉토리: {data_dir}")
print(f"디렉토리 존재: {data_dir.exists()}")

if data_dir.exists():
    files = list(data_dir.glob("*"))
    print(f"\n파일 목록 ({len(files)}개):")
    for file in files:
        print(f"  - {file.name} ({file.stat().st_size:,} bytes)")
else:
    print("데이터 디렉토리가 없습니다!")

데이터 디렉토리: /Users/kimjunghyeon/Desktop/workspace/ai-agent/data/documents
디렉토리 존재: True

파일 목록 (3개):
  - 소득세법.doc (1,965,420 bytes)
  - 전국_야영장_20260106.csv (1,074,261 bytes)
  - 소득세법.pdf (794,133 bytes)


In [4]:
# 지원되는 파일 형식 확인
print("지원되는 MIME 타입:")
for mime_type in ParserFactory.get_supported_mime_types():
    print(f"  - {mime_type}")

print("\n지원되는 파일 확장자:")
for ext in ParserFactory.get_supported_extensions():
    print(f"  - {ext}")

지원되는 MIME 타입:
  - application/pdf
  - application/vnd.openxmlformats-officedocument.wordprocessingml.document
  - application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
  - text/csv
  - application/csv
  - application/msword
  - application/vnd.ms-word

지원되는 파일 확장자:
  - .pdf
  - .docx
  - .xlsx
  - .csv
  - .doc


In [5]:
# MIME 타입 매핑
mime_mapping = {
    ".pdf": "application/pdf",
    ".docx": "application/vnd.openxmlformats-officedocument.wordprocessingml.document",
    ".xlsx": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
    ".csv": "text/csv",  # CSV는 아직 지원하지 않지만 테스트용
    ".txt": "text/plain"
}

def get_mime_type(file_path):
    """파일 확장자로부터 MIME 타입 추정"""
    suffix = Path(file_path).suffix.lower()
    return mime_mapping.get(suffix, "unknown")

In [6]:
# 개별 파서 테스트 함수
async def test_parser(file_path, mime_type=None):
    """개별 파서 테스트"""
    file_path = Path(file_path)
    if not file_path.exists():
        print(f"❌ 파일이 존재하지 않습니다: {file_path}")
        return None
    
    if mime_type is None:
        mime_type = get_mime_type(file_path)
    
    print(f"\n📄 테스트: {file_path.name}")
    print(f"   MIME 타입: {mime_type}")
    print(f"   파일 크기: {file_path.stat().st_size:,} bytes")
    
    # 지원 여부 확인
    if not ParserFactory.is_supported_mime_type(mime_type):
        print(f"   ⚠️ 지원하지 않는 파일 형식입니다.")
        return None
    
    try:
        # 파서 생성 및 파싱
        parser = ParserFactory.get_parser(mime_type)
        print(f"   🔧 파서: {parser.__class__.__name__}")
        
        # 파싱 실행
        parsed_content = await parser.parse(str(file_path))
        
        # 결과 출력
        print(f"   ✅ 파싱 성공!")
        print(f"   📝 텍스트 길이: {len(parsed_content.raw_text):,} 문자")
        print(f"   📊 테이블 수: {len(parsed_content.tables)}개")
        print(f"   🖼️ 이미지 수: {len(parsed_content.images)}개")
        print(f"   📋 메타데이터: {len(parsed_content.metadata)}개 항목")
        
        # 텍스트 미리보기
        preview = parsed_content.raw_text[:300].replace('\n', ' ').strip()
        if len(parsed_content.raw_text) > 300:
            preview += "..."
        print(f"   👀 텍스트 미리보기: {preview}")
        
        # 테이블 정보
        if parsed_content.tables:
            print(f"   📊 테이블 상세:")
            for i, table in enumerate(parsed_content.tables[:3]):  # 최대 3개만
                print(f"      - 테이블 {i+1}: {len(table.headers)}열 x {len(table.rows)}행")
                if table.headers:
                    headers_preview = ", ".join(table.headers[:5])
                    if len(table.headers) > 5:
                        headers_preview += "..."
                    print(f"        헤더: {headers_preview}")
        
        return parsed_content
        
    except Exception as e:
        print(f"   ❌ 파싱 실패: {e}")
        return None

In [7]:
# DocumentProcessor를 통한 테스트
async def test_document_processor(file_path, mime_type=None):
    """DocumentProcessor를 통한 통합 테스트"""
    file_path = Path(file_path)
    if not file_path.exists():
        print(f"❌ 파일이 존재하지 않습니다: {file_path}")
        return None
    
    if mime_type is None:
        mime_type = get_mime_type(file_path)
    
    print(f"\n🔄 DocumentProcessor 테스트: {file_path.name}")
    
    try:
        processor = DocumentProcessor()
        
        # 텍스트 추출
        extracted_text = await processor.extract_text_from_file(str(file_path), mime_type)
        print(f"   ✅ 텍스트 추출 성공!")
        print(f"   📝 추출된 텍스트 길이: {len(extracted_text):,} 문자")
        
        # 텍스트 유효성 검증
        is_valid = processor.validate_file_content(extracted_text)
        print(f"   🔍 텍스트 유효성: {'✅ 유효' if is_valid else '❌ 무효'}")
        
        # 청킹 테스트
        if is_valid:
            chunks = processor.chunk_text(extracted_text, {"file_name": file_path.name})
            print(f"   🧩 청킹 결과: {len(chunks)}개 청크")
            
            if chunks:
                avg_size = sum(chunk['char_count'] for chunk in chunks) / len(chunks)
                avg_tokens = sum(chunk['token_count'] for chunk in chunks) / len(chunks)
                print(f"   📊 평균 청크 크기: {avg_size:.0f}문자, {avg_tokens:.0f}토큰")
                
                # 첫 번째 청크 미리보기
                first_chunk_preview = chunks[0]['content'][:200].replace('\n', ' ').strip()
                if len(chunks[0]['content']) > 200:
                    first_chunk_preview += "..."
                print(f"   👀 첫 청크 미리보기: {first_chunk_preview}")
        
        return extracted_text
        
    except Exception as e:
        print(f"   ❌ DocumentProcessor 실패: {e}")
        return None

In [8]:
# 모든 파일 테스트 실행
async def test_all_files():
    """데이터 디렉토리의 모든 파일 테스트"""
    if not data_dir.exists():
        print("데이터 디렉토리가 없습니다!")
        return
    
    files = [f for f in data_dir.glob("*") if f.is_file()]
    print(f"🚀 총 {len(files)}개 파일 테스트 시작\n")
    print("=" * 80)
    
    results = {}
    
    for file_path in files:
        mime_type = get_mime_type(file_path)
        
        # 개별 파서 테스트
        parsed_result = await test_parser(file_path, mime_type)
        
        # DocumentProcessor 테스트
        processor_result = await test_document_processor(file_path, mime_type)
        
        results[file_path.name] = {
            'parsed': parsed_result is not None,
            'processed': processor_result is not None,
            'mime_type': mime_type
        }
        
        print("\n" + "="*80)
    
    # 결과 요약
    print("\n📊 테스트 결과 요약:")
    print("-" * 60)
    
    success_count = 0
    for filename, result in results.items():
        status = "✅ 성공" if result['processed'] else "❌ 실패"
        if result['processed']:
            success_count += 1
        print(f"{filename:<30} {result['mime_type']:<15} {status}")
    
    print(f"\n성공률: {success_count}/{len(files)} ({success_count/len(files)*100:.1f}%)")
    
    return results

# 테스트 실행
test_results = await test_all_files()

텍스트 추출 실패 (/Users/kimjunghyeon/Desktop/workspace/ai-agent/data/documents/소득세법.doc): 지원하지 않는 파일 형식: unknown


🚀 총 3개 파일 테스트 시작


📄 테스트: 소득세법.doc
   MIME 타입: unknown
   파일 크기: 1,965,420 bytes
   ⚠️ 지원하지 않는 파일 형식입니다.

🔄 DocumentProcessor 테스트: 소득세법.doc
   ❌ DocumentProcessor 실패: 지원하지 않는 파일 형식: unknown


📄 테스트: 전국_야영장_20260106.csv
   MIME 타입: text/csv
   파일 크기: 1,074,261 bytes
   🔧 파서: CSVParser
   ✅ 파싱 성공!
   📝 텍스트 길이: 21,868 문자
   📊 테이블 수: 1개
   🖼️ 이미지 수: 0개
   📋 메타데이터: 5개 항목
   👀 텍스트 미리보기: 번호 | 야영장명 | 사업주체_구분 | 도 | 시군구 | 주소 | 주요시설 일반야영장 | 주요시설 자동차야영장 | 주요시설 글램핑 | 주요시설 카라반 | 주요시설 개인 카라반 | 주요시설 덤프스테이션 | 사이트 크기1 가로 | 사이트 크기2 가로 | 사이트 크기3 가로 | 사이트 크기1 세로 | 사이트 크기2 세로 | 사이트 크기3 세로 | 사이트 크기1 수량 | 사이트 크기2 수량 | 사이트 크기3 수량 | 인허가일자 | 화로대 | 부대시설 | 주변이용가능시설 | 소화기 개수 | 방화수 개수 | 방화사 개수 | 화재감지기 개수 |...
   📊 테이블 상세:
      - 테이블 1: 32열 x 5106행
        헤더: 번호, 야영장명, 사업주체_구분, 도, 시군구...

🔄 DocumentProcessor 테스트: 전국_야영장_20260106.csv
   ✅ 텍스트 추출 성공!
   📝 추출된 텍스트 길이: 21,868 문자
   🔍 텍스트 유효성: ✅ 유효
   🧩 청킹 결과: 25개 청크
   📊 평균 청크 크기: 970문자, 539토큰
   👀 첫 청크 미리보기: 번호 | 야영장명 | 사업주체_구분 | 도 | 시군구 | 주소 | 주요시설 일반야영장 | 주요시설 자동차야영

PyMuPDF 파싱 실패: PyMuPDF PDF 파싱 실패: document closed. PyPDF2로 fallback합니다.


   ✅ 파싱 성공!
   📝 텍스트 길이: 271,032 문자
   📊 테이블 수: 0개
   🖼️ 이미지 수: 0개
   📋 메타데이터: 6개 항목
   👀 텍스트 미리보기: [페이지 1] 법제처                                                            1                                                       국가법령정보센터 소득세법    소득세법  [시행 2026. 1. 2.] [법률 제21065호, 2025. 10. 1., 타법개정]  재정경제부 (재산세제과(양도소득세)) 044-215-4312  재정경제부 (소득세제과(근로소득)) 044-215-4216  재정경제부 (금융세제과(이자소득, 배당소득)) 044-...

🔄 DocumentProcessor 테스트: 소득세법.pdf


PyMuPDF 파싱 실패: PyMuPDF PDF 파싱 실패: document closed. PyPDF2로 fallback합니다.


   ✅ 텍스트 추출 성공!
   📝 추출된 텍스트 길이: 271,032 문자
   🔍 텍스트 유효성: ✅ 유효
   🧩 청킹 결과: 302개 청크
   📊 평균 청크 크기: 993문자, 548토큰
   👀 첫 청크 미리보기: [페이지 1] 법제처                                                            1                                                       국가법령정보센터 소득세법    소득세법  [시행 2026. 1. 2.] [법률 제21065호, 2025. 10. 1., 타법개정]...


📊 테스트 결과 요약:
------------------------------------------------------------
소득세법.doc                       unknown         ❌ 실패
전국_야영장_20260106.csv            text/csv        ✅ 성공
소득세법.pdf                       application/pdf ✅ 성공

성공률: 2/3 (66.7%)


In [9]:
# CSV 파일을 위한 간단한 파서 (필요한 경우)
# CSV는 기본 지원하지 않으므로 간단히 텍스트로 읽어보기

def test_csv_file_simple(file_path):
    """CSV 파일 간단 테스트 (텍스트로 읽기)"""
    try:
        import pandas as pd
        
        print(f"\n📊 CSV 파일 테스트: {Path(file_path).name}")
        
        # pandas로 읽기
        df = pd.read_csv(file_path)
        print(f"   📏 형태: {df.shape[0]}행 x {df.shape[1]}열")
        print(f"   📋 컬럼: {list(df.columns[:5])}..." if len(df.columns) > 5 else f"   📋 컬럼: {list(df.columns)}")
        
        # 텍스트로 변환
        text_content = df.to_string(max_rows=20)
        print(f"   📝 텍스트 길이: {len(text_content):,} 문자")
        
        # 미리보기
        print(f"   👀 데이터 미리보기:")
        print(df.head(3).to_string())
        
        return text_content
        
    except Exception as e:
        print(f"   ❌ CSV 테스트 실패: {e}")
        return None

# CSV 파일 찾아서 테스트
csv_files = list(data_dir.glob("*.csv"))
if csv_files:
    for csv_file in csv_files:
        test_csv_file_simple(csv_file)
else:
    print("CSV 파일을 찾을 수 없습니다.")


📊 CSV 파일 테스트: 전국_야영장_20260106.csv
   ❌ CSV 테스트 실패: 'utf-8' codec can't decode byte 0xb9 in position 0: invalid start byte


In [10]:
# 특정 파일 상세 분석 (선택사항)
# 테스트 결과에서 성공한 파일 중 하나를 선택하여 상세 분석

async def detailed_analysis(file_path):
    """특정 파일의 상세 분석"""
    file_path = Path(file_path)
    mime_type = get_mime_type(file_path)
    
    print(f"\n🔍 상세 분석: {file_path.name}")
    print("=" * 50)
    
    if ParserFactory.is_supported_mime_type(mime_type):
        parser = ParserFactory.get_parser(mime_type)
        parsed_content = await parser.parse(str(file_path))
        
        print(f"📋 메타데이터 상세:")
        for key, value in parsed_content.metadata.items():
            print(f"   {key}: {value}")
        
        print(f"\n🏗️ 구조 정보:")
        for key, value in parsed_content.structure.items():
            print(f"   {key}: {value}")
        
        if parsed_content.tables:
            print(f"\n📊 테이블 상세 정보:")
            for i, table in enumerate(parsed_content.tables):
                print(f"   테이블 {i+1}:")
                print(f"     - 크기: {len(table.headers)}열 x {len(table.rows)}행")
                print(f"     - 헤더: {table.headers}")
                print(f"     - 위치: {table.position}")
                print(f"     - 메타데이터: {table.metadata}")
                if table.rows:
                    print(f"     - 첫 번째 행: {table.rows[0]}")
                print()
        
        return parsed_content
    else:
        print(f"지원하지 않는 파일 형식입니다: {mime_type}")
        return None

# 성공한 파일 중 첫 번째 파일로 상세 분석 (있는 경우)
if test_results:
    successful_files = [filename for filename, result in test_results.items() if result['processed']]
    if successful_files:
        selected_file = data_dir / successful_files[0]
        await detailed_analysis(selected_file)
    else:
        print("성공적으로 파싱된 파일이 없습니다.")


🔍 상세 분석: 전국_야영장_20260106.csv
📋 메타데이터 상세:
   parser: csv_parser
   encoding: utf-8
   total_rows: 5106
   total_columns: 32
   file_size: 753078

🏗️ 구조 정보:
   total_rows: 5106
   total_columns: 32
   headers: ['번호', '야영장명', '사업주체_구분', '도', '시군구', '주소', '주요시설 일반야영장', '주요시설 자동차야영장', '주요시설 글램핑', '주요시설 카라반', '주요시설 개인 카라반', '주요시설 덤프스테이션', '사이트 크기1 가로', '사이트 크기2 가로', '사이트 크기3 가로', '사이트 크기1 세로', '사이트 크기2 세로', '사이트 크기3 세로', '사이트 크기1 수량', '사이트 크기2 수량', '사이트 크기3 수량', '인허가일자', '화로대', '부대시설', '주변이용가능시설', '소화기 개수', '방화수 개수', '방화사 개수', '화재감지기 개수', '테마환경', '캠핑장비대여', '반려동물출입']
   data_types: {'번호': 'numeric', '야영장명': 'text', '사업주체_구분': 'text', '도': 'text', '시군구': 'text', '주소': 'date', '주요시설 일반야영장': 'numeric', '주요시설 자동차야영장': 'numeric', '주요시설 글램핑': 'numeric', '주요시설 카라반': 'numeric', '주요시설 개인 카라반': 'numeric', '주요시설 덤프스테이션': 'numeric', '사이트 크기1 가로': 'numeric', '사이트 크기2 가로': 'numeric', '사이트 크기3 가로': 'numeric', '사이트 크기1 세로': 'numeric', '사이트 크기2 세로': 'numeric', '사이트 크기3 세로': 'numeric', '사이트 크기1 수량': 'numeric'